In [60]:
!pip install pillow   

from PIL import Image
import os, glob

import tensorflow as tf
from tensorflow import keras
import numpy as np


# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/아이펠실습/train/scissor"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")

image_dir_path = os.getenv("HOME") + "/아이펠실습/train/rock"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")
    
print("바위 이미지 resize 완료!")
    
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서

image_dir_path = os.getenv("HOME") + "/아이펠실습/train/paper"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("보 이미지 resize 완료!")   
    


def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data= 2320   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels


image_dir_path = os.getenv("HOME") + "/아이펠실습/train"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

n_channel_1= 32
n_channel_2= 48
n_dense= 128
n_train_epoch= 30



model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train, epochs=n_train_epoch)



def load_test(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=2320   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("테스트 데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels



image_dir_path = os.getenv("HOME") + "/아이펠실습/rock_scissor_paper"
(x_test, y_test)=load_test(image_dir_path)




x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0



# 모델 훈련
model.evaluate(x_test_norm, y_test, batch_size = 126)


You should consider upgrading via the '/home/aiffel/anaconda3/bin/python3 -m pip install --upgrade pip' command.
이미지 디렉토리 경로:  /home/aiffel/아이펠실습/train/scissor
가위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/아이펠실습/train/rock
바위 이미지 resize 완료!
이미지 디렉토리 경로:  /home/aiffel/아이펠실습/train/paper
보 이미지 resize 완료!
학습데이터(x_train)의 이미지 개수는 2320 입니다.
x_train shape: (2320, 28, 28, 3)
y_train shape: (2320,)
Epoch 1/30
73/73 [==============================] - 0s 1ms/step - loss: 1.0861 - accuracy: 0.3767
Epoch 2/30
73/73 [==============================] - 0s 1ms/step - loss: 1.0153 - accuracy: 0.4759
Epoch 3/30
73/73 [==============================] - 0s 1ms/step - loss: 0.9338 - accuracy: 0.5457
Epoch 4/30
73/73 [==============================] - 0s 1ms/step - loss: 0.8672 - accuracy: 0.5931
Epoch 5/30
73/73 [==============================] - 0s 1ms/step - loss: 0.8166 - accuracy: 0.6319
Epoch 6/30
73/73 [==============================] - 0s 1ms/step - loss: 0.7383 - accuracy: 0.6728
Epoch 7/30
73/73 [==

[0.49189773201942444, 0.9512931108474731]

우선 여러 데이터를 넣어본 결과, 학습용 데이터는 많으면 많을수록 좋다는것을 깨달았습니다. 
KAGGLE  에서 ROCK SCISSOR PAPER JPG 데이터 파일을 검색해 학습용 데이터로 사용하였습니다.
반복횟수 (Epoch값) 의 경우에는 너무 많이 하면 overfitting 이 나서 오히려 정확도가 떨어졌습니다.
반복횟수는 30회 정도가 제가 구한 데이터와 잘 맞았습니다.
총 2320 개라는 많은량의 사진이라 dense 및 필터값 또한 높혀주었습니다.

정확도가 높게 나온이유는 아무래도 학습용 데이터가 워낙 방대했고,
제가 테스트용으로 쓴 데이터 또한 배경의 군더더기가 없이 깔끔한 편이라 잘 들어맞았던것 같습니다.
